In [ ]:
# Improve the original answer and generate a better answer 

In [ ]:
import pandas as pd
import google.generativeai as genai
import dotenv
import time
import os
dotenv.load_dotenv()

True

In [ ]:
raw_dataset = pd.read_csv('../dataset/cleaned_dataset.csv')
raw_dataset.head

<bound method NDFrame.head of                                               Question  \
0                              How do I apply for CPT?   
1                 What documents are required for CPT?   
2                   How long does CPT processing take?   
3       Can I work before receiving CPT authorization?   
4                   Do I need to pay any fees for CPT?   
..                                                 ...   
126  How do I check the status of my Form I-765 app...   
127  What should I do if I encounter technical issu...   
128  Can I file Form I-765 online if I am applying ...   
129  How do I upload supporting documents when fili...   
130  What should I do if I realize I made a mistake...   

                                                Answer  
0    You must get approval from your Designated Sch...  
1    Typically you need:\n- CPT Request Form from y...  
2    Processing usually takes 5-10 business days af...  
3    No you must wait for approval and receiv

In [5]:
raw_dataset['Answer'].iloc[0]

'You must get approval from your Designated School Official (DSO) and submit a CPT request form along with an offer letter from your employer.'

In [6]:
class DatasetAnsImprover:
    def __init__(self) -> None:
        api_key = os.getenv("GOOGLE_API_KEY")
        model_name = 'gemini-2.0-flash-exp'
        genai.configure(api_key=api_key)
        improve_answers_prompt = """
                            You are a leagal expert advisor. You will be given a question and an answer. You are required to improve the answer. Just reply with improved answer nothing else. Answer in a single paragraph in 3-4 sentences"""
        self.chat_model = genai.GenerativeModel(model_name, system_instruction=improve_answers_prompt)

    def improve_answer(self, question: str, answer:str) -> bool:
        prompt = f"""
        Question: {question}
        Answer: {answer}
        Improved Answer:        
        """
        response = self.chat_model.generate_content(prompt)
        return response.text.strip()


In [ ]:
# import pprint
# dataset_improver = DatasetAnsImprover()
# improved_answer = dataset_improver.improve_answer(raw_dataset['Question'].iloc[0], raw_dataset['Answer'].iloc[0])

# print('Original Answer:', raw_dataset['Answer'].iloc[0])
# print('Improved Answer:')
# pprint.pprint(improved_answer)

Original Answer: You must get approval from your Designated School Official (DSO) and submit a CPT request form along with an offer letter from your employer.
Improved Answer:
('To apply for CPT, start by obtaining a job offer that qualifies for CPT and '
 'then completing the CPT request form provided by your school. Next, you must '
 'secure approval from your Designated School Official (DSO), who will review '
 'your application and ensure it meets all requirements. Finally, submit the '
 'completed CPT request form, along with your offer letter and any other '
 'required documentation, to your DSO for processing and authorization.')


In [ ]:
dataset_improver = DatasetAnsImprover()

# loop over the dataset and improve the answers
for index, row in raw_dataset.iterrows():
    improved_answer = dataset_improver.improve_answer(row['Question'], row['Answer'])
    time.sleep(3)
    if index % 10 == 0:
        time.sleep(5)
    raw_dataset.at[index, 'Answer'] = improved_answer

raw_dataset.to_csv('../dataset/cleaned_dataset_improved.csv', index=False)